In [1]:
import dotenv
import os

dotenv.load_dotenv()
# List all keys in the .env file
import os
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")


In [2]:
import pandas as pd

df = pd.read_excel('/home/arantesvictorl/repositorios/trabalho/evah/agente-jair-autorizacao/justificador/data/Manual de instruções.xlsx')
df["nivel1_conteudo"] = df["nivel_1"] + "-" + df["conteudo"]

In [3]:
df

,filename,nivel_1,nivel_2,conteudo,nivel1_conteudo
0,manual de instruções,ALERGIA E IMUNOLOGIA,Imunoterapia alérgeno específica,Imunoterapia alérgeno específica\nA imunoterap...,ALERGIA E IMUNOLOGIA-Imunoterapia alérgeno esp...
1,manual de instruções,ALERGIA E IMUNOLOGIA,Provas imunoalérgicas,Provas imunoalérgicas\nOs procedimentos provas...,ALERGIA E IMUNOLOGIA-Provas imunoalérgicas\nOs...
2,manual de instruções,ALERGIA E IMUNOLOGIA,Teste de contato,Teste de contato\nMétodo mais eficiente para c...,ALERGIA E IMUNOLOGIA-Teste de contato\nMétodo ...
3,manual de instruções,ALERGIA E IMUNOLOGIA,Testes cutâneo alérgicos de leitura imediata,Testes cutâneo alérgicos de leitura imediata\n...,ALERGIA E IMUNOLOGIA-Testes cutâneo alérgicos ...
4,manual de instruções,ANATOMIA PATOLÓGICA,Citopatológico,Citopatológico\nCódigo\nDescrição\n40601137\nP...,ANATOMIA PATOLÓGICA-Citopatológico\nCódigo\nDe...
...,...,...,...,...,...
628,manual de instruções,UROLOGIA,Tratamento da hiperatividade vesical,Tratamento da hiperatividade vesical\nCódigo\n...,UROLOGIA-Tratamento da hiperatividade vesical\...
629,manual de instruções,UROLOGIA,Ureterolitotomia ou ureterolitotripsia transur...,Ureterolitotomia ou ureterolitotripsia transur...,UROLOGIA-Ureterolitotomia ou ureterolitotripsi...
630,manual de instruções,UROLOGIA,Ureterotomia endoscópica,Ureterotomia endoscópica\nCódigo\nDescrição\n3...,UROLOGIA-Ureterotomia endoscópica\nCódigo\nDes...
631,manual de instruções,UROLOGIA,Uretrotomia interna,Uretrotomia interna\nCódigo\nDescrição\n311042...,UROLOGIA-Uretrotomia interna\nCódigo\nDescriçã...


In [4]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content=row["nivel1_conteudo"], 
        metadata={
            "filename": row["filename"],
            "nivel_1": row["nivel_1"],
            "nivel_2": row["nivel_2"]
        }
    )
    for _, row in df.iterrows()
]


In [5]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [6]:
from langchain_qdrant import FastEmbedSparse, RetrievalMode

sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

/home/arantesvictorl/miniconda3/envs/agente-jair-autorizacao/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 29 files: 100%|██████████| 29/29 [00:02<00:00, 11.26it/s]


In [8]:
from langchain_qdrant import QdrantVectorStore

url = "https://059372fd-0731-4343-a57e-e613adc7c768.us-west-2-0.aws.cloud.qdrant.io:6333/"

qdrant = QdrantVectorStore.from_documents(
    documents,
    embeddings,
    sparse_embedding=sparse_embeddings,
    url=url,
    prefer_grpc=True,
    api_key=QDRANT_API_KEY,
    collection_name="documents_jair",
    retrieval_mode=RetrievalMode.HYBRID
)

usando colection

In [9]:
def initialize_qdrant_vector_store():
    qdrant = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    collection_name="documents_jair",
    api_key=QDRANT_API_KEY,
    url="https://059372fd-0731-4343-a57e-e613adc7c768.us-west-2-0.aws.cloud.qdrant.io:6333/",)
    return qdrant

qdrant = initialize_qdrant_vector_store()

In [10]:
results = qdrant.similarity_search_with_score(
    "Dor na coluna", k=2
)
for res in results:
    print(f"* {res[0].page_content} [{res[0].metadata}]")

* TERAPIA DA DOR E ACUPUNTURA-Segmento em coluna vertebral
Segmento vertebral

em coluna vertebral:
refere-se a uma unidade motora de movimento, composta
de duas vértebras, um disco intervertebral e estruturas cápsulo-ligamentares e musculares a elas
vinculadas.
Segmento anatômico em coluna vertebral:
refere-se à divisão da coluna em segmentos cervical,
dorsal (ou torácico), lombar e sacrococcígeo, não podendo ser usado outras unidades por similaridade
(ex. com o osso ilíaco).
As codificações em cirurgia de coluna podem ser remuneradas por vértebra, por segmento vertebral,
por segmento anatômico de coluna ou por cirurgia. [{'nivel_1': 'TERAPIA DA DOR E ACUPUNTURA', 'nivel_2': 'Segmento em coluna vertebral', 'filename': 'manual de instruções', '_id': 'c2780516-611d-4a43-a2bf-172f57cbcb44', '_collection_name': 'documents_jair'}]
* TERAPIA DA DOR E ACUPUNTURA-Discografia provocativa
Código
Descrição
40814106
Discografia
A discografia é uma modalidade puramente diagnóstica e deve ser indic

In [ ]:
results